![MLU Logo](../../data/MLU_Logo.png)

# <a name="0">Responsible ML - Final Project</a>

Build a fair [LogisticRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) that predicts the __credit_risk__ field (whether loan is repaid or not) of the [German Credit Dataset](https://archive.ics.uci.edu/ml/datasets/South+German+Credit+%28UPDATE%29).

### Final Project Problem: Loan Approval

__Problem Definition__:
Given a set of features about an individual (e.g. age, past credit history, immigration status, ...) predict whether a loan is repaid or not (is customer a credit risk). We impose the additional constraint that the model should be fair with respect to different age groups ($\geq$ 25y and $<$ 25y).
, in the banking industry, there are certain patterns that could be illegal. One of illegal patterns would be age playing a significant role (loans should be approved/denied regardless of an individuals age).

For example, certain laws declare it unlawful for creditors to discriminate against any applicant on the basis of age (or other sensitive attributes). For more details, have a look at this paper:

``` 
F. Kamiran and T. Calders, "Data Preprocessing Techniques for Classification without Discrimination," Knowledge and Information Systems, 2012
```

1. <a href="#1">Read the datasets</a> (Given) 
2. <a href="#2">Data Processing</a> (Implement)
    * <a href="#21">Exploratory Data Analysis</a>
    * <a href="#22">Select features to build the model</a> (Suggested)
    * <a href="#23">Train - Validation - Test Datasets</a>
    * <a href="#24">Data Processing with Pipeline</a>
3. <a href="#3">Train (and Tune) a Classifier on the Training Dataset</a> (Implement)
4. <a href="#4">Make Predictions on the Test Dataset</a> (Implement)
5. <a href="#5">Evaluate Results</a> (Given)


__Datasets and Files:__


- ```german_credit_training.csv```: Training data with loan applicants features, credit history, dependents, savings, account status, age group (and more). The label is __credit_risk__.

- ```german_credit_test.csv```: Test data with same features as above apart from label. This will be the data to make predictions for to emulate a production environment.

In [27]:
# Reshaping/basic libraries
import pandas as pd
import numpy as np

# Plotting libraries
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("darkgrid", {"axes.facecolor": ".9"})

# ML libraries
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from aif360.datasets import BinaryLabelDataset, Dataset
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.algorithms.preprocessing import DisparateImpactRemover

# Operational libraries
import sys
sys.path.append("..")

# Jupyter(lab) libraries
import warnings
warnings.filterwarnings("ignore")

## 1. <a name="1">Read the datasets</a> (Given)
(<a href="#0">Go to top</a>)

Then, we read the __training__ and __test__ datasets into dataframes, using [Pandas](https://pandas.pydata.org/pandas-docs/stable/user_guide/10min.html). This library allows us to read and manipulate our data.

In [3]:
training_data = pd.read_csv('../../data/final_project/german_credit_training.csv')
test_data = pd.read_csv('../../data/final_project/german_credit_test.csv')

print('The shape of the training dataset is:', training_data.shape)
print('The shape of the test dataset is:', test_data.shape)

The shape of the training dataset is: (800, 24)
The shape of the test dataset is: (200, 23)


## 2. <a name="2">Data Processing</a> (Implement)
(<a href="#0">Go to top</a>) 

### 2.1 <a name="21">Exploratory Data Analysis</a>
(<a href="#2">Go to Data Processing</a>)

We look at number of rows, columns, and some simple statistics of the datasets.

In [4]:
training_data.head()

,ID,sex,personal_status,checking_account_status,credit_duration_months,credit_history,credit_purpose,credit_amount,savings,employed_since_years,...,age_years,other_installment_plans,housing,num_existing_credits,job_status,num_people_liable_for,telephone,foreign_worker,credit_risk,age_groups
0,564,male,married/widowed,... < 0 DM,24,existing credits paid back duly till now,retraining,4712,.. >= 1000 DM,1 <= ... < 4 years,...,37,bank,rent,2-3,management/self-employed/highly qualified empl...,0 to 2,True,False,0,1
1,484,male,married/widowed,... >= 200 DM,12,all credits at this bank paid back duly,others,1163,100 <= ... < 500 DM,1 <= ... < 4 years,...,44,none,rent,1,skilled employee/official,0 to 2,True,False,0,1
2,26,female,single,... >= 200 DM,6,delay in paying off in the past,furniture/equipment,426,unknown/no savings account,.. >= 7 years,...,39,none,rent,1,unskilled - resident,0 to 2,False,False,0,1
3,443,male,married/widowed,... >= 200 DM,12,no credits taken/all credits paid back duly,repairs,719,unknown/no savings account,.. >= 7 years,...,41,bank,rent,1,unskilled - resident,3 or more,False,False,1,1
4,211,female/male,female (non-single) or male (single),... >= 200 DM,36,no credits taken/all credits paid back duly,furniture/equipment,3835,.. >= 1000 DM,.. >= 7 years,...,45,none,rent,1,unskilled - resident,0 to 2,True,False,0,1


In [5]:
test_data.head()

,ID,sex,personal_status,checking_account_status,credit_duration_months,credit_history,credit_purpose,credit_amount,savings,employed_since_years,...,property,age_years,other_installment_plans,housing,num_existing_credits,job_status,num_people_liable_for,telephone,foreign_worker,age_groups
0,963,male,married/widowed,... >= 200 DM,24,no credits taken/all credits paid back duly,furniture/equipment,2397,100 <= ... < 500 DM,.. >= 7 years,...,building society savings agreement/life insurance,35,bank,rent,2-3,skilled employee/official,0 to 2,True,False,1
1,611,female/male,female (non-single) or male (single),0 <= ... < 200 DM,10,no credits taken/all credits paid back duly,others,1240,... < 100 DM,.. >= 7 years,...,real estate,48,none,own,1,unskilled - resident,3 or more,False,False,1
2,106,male,married/widowed,... >= 200 DM,18,critical account/other credits existing (not a...,others,6458,unknown/no savings account,.. >= 7 years,...,real estate,39,bank,rent,2-3,management/self-employed/highly qualified empl...,3 or more,True,False,1
3,891,male,married/widowed,... >= 200 DM,15,all credits at this bank paid back duly,furniture/equipment,1829,unknown/no savings account,.. >= 7 years,...,building society savings agreement/life insurance,46,none,rent,2-3,skilled employee/official,0 to 2,True,False,1
4,342,female,single,... < 0 DM,18,no credits taken/all credits paid back duly,furniture/equipment,3213,100 <= ... < 500 DM,... < 1 year,...,unknown/no property,25,none,for free,1,skilled employee/official,0 to 2,False,False,1


In [6]:
# Implement more EDA here


In [7]:
training_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 24 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   ID                        800 non-null    int64 
 1   sex                       800 non-null    object
 2   personal_status           800 non-null    object
 3   checking_account_status   800 non-null    object
 4   credit_duration_months    800 non-null    int64 
 5   credit_history            800 non-null    object
 6   credit_purpose            800 non-null    object
 7   credit_amount             800 non-null    int64 
 8   savings                   800 non-null    object
 9   employed_since_years      800 non-null    object
 10  installment_rate          800 non-null    object
 11  other_debtors_guarantors  800 non-null    object
 12  present_residence_since   800 non-null    object
 13  property                  800 non-null    object
 14  age_years                 

### 2.2 <a name="22">Select features to build the model</a> 
(<a href="#2">Go to Data Processing</a>)

Let's use all the features today. Below you see a snippet of code that separates categorical and numerical columns based on their data type. This should only be used if we are sure that the data types are correctly assigned (check during EDA). Mindful with some of the feature names - they suggest numerical values but upon inspection it should become clear that they are actually categoricals (e.g. `employed_since_years` has been binned into groups).


In [8]:
# Grab model features/inputs and target/output
categorical_features = training_data.drop('credit_risk',axis = 1).select_dtypes(include='object').columns.tolist()
print("Categorical columns:", categorical_features)

print("")

numerical_features = training_data.drop('credit_risk',axis = 1).select_dtypes(include=np.number).columns.tolist()
print("Numerical columns:", numerical_features)

Categorical columns: ['sex', 'personal_status', 'checking_account_status', 'credit_history', 'credit_purpose', 'savings', 'employed_since_years', 'installment_rate', 'other_debtors_guarantors', 'present_residence_since', 'property', 'other_installment_plans', 'housing', 'num_existing_credits', 'job_status', 'num_people_liable_for']

Numerical columns: ['ID', 'credit_duration_months', 'credit_amount', 'age_years', 'age_groups']


We notice that `ID` is identified as numerical column. ID's should never be used as features for training as they are unique by row. Let's drop the ID from the model features after we have separated target and features:

In [9]:
model_target = "credit_risk"
model_features = categorical_features + numerical_features

print("Model features: ", model_features)
print("\n")
print("Model target: ", model_target)

Model features:  ['sex', 'personal_status', 'checking_account_status', 'credit_history', 'credit_purpose', 'savings', 'employed_since_years', 'installment_rate', 'other_debtors_guarantors', 'present_residence_since', 'property', 'other_installment_plans', 'housing', 'num_existing_credits', 'job_status', 'num_people_liable_for', 'ID', 'credit_duration_months', 'credit_amount', 'age_years', 'age_groups']


Model target:  credit_risk


In [10]:
to_remove = "ID"

# Drop 'ID' feature from the respective list(s)
if to_remove in model_features:
    model_features.remove(to_remove)
if to_remove in categorical_features:
    categorical_features.remove(to_remove)
if to_remove in numerical_features:
    numerical_features.remove(to_remove)

### 2.3 <a name="23">Feature transformation</a>
(<a href="#2">Go to Data Processing</a>)

Here, you have different options. You can use Reweighing, Disparate Impact Remover or Suppression. 

In [11]:
# To use DI (with AIF360 library), we need to:
# 1. Fill all gaps (deal with missing values)
# 2. Convert all features in our dataset to numerical values (using OneHotEncoding)
# 3. Fit the transformer to training dataset and transform all datase splits


### STEP 1 ###
##############

# Preprocess the numerical features
numerical_processor = Pipeline(
    [("num_imputer", SimpleImputer(strategy="mean")), ("num_scaler", MinMaxScaler())]
)
# Preprocess the categorical features
categorical_processor = Pipeline(
    [
        ("cat_imputer", SimpleImputer(strategy="constant", fill_value="missing")),
        ("cat_encoder", OneHotEncoder(handle_unknown="ignore")),
    ]
)

### STEP 2 ###
##############

# Combine all data preprocessors from above
data_processor = ColumnTransformer(
    [
        ("numerical_processing", numerical_processor, numerical_features),
        ("categorical_processing", categorical_processor, categorical_features),
    ]
)

In [12]:
# Fit the dataprocessor to our training data and apply transform to test data
processed_train = data_processor.fit_transform(training_data[model_features])
processed_test = data_processor.transform(test_data)

In [13]:
# Extract feature names from data processor (we need this as one-hot encoding creates new columns)
cat_ft_names = list(data_processor.transformers_[1][1].named_steps['cat_encoder'].get_feature_names(categorical_features))

# Convert the sparese matrix to a pandas dataframe and name columns
processed_train_df = pd.DataFrame.sparse.from_spmatrix(processed_train, columns = numerical_features+cat_ft_names)
processed_test_df = pd.DataFrame.sparse.from_spmatrix(processed_test, columns = numerical_features+cat_ft_names)

# Join label with the trainsformed dataset so we can pass it to AIF360
train_df = pd.concat([processed_train_df, training_data[model_target]], axis = 1)

In [15]:
# Create a Dataset construct for AIF360
binaryLabelDataset_train = BinaryLabelDataset(
    df=train_df,
    label_names=["credit_risk"],
    protected_attribute_names=["age_groups"],
    favorable_label=0.0,
    unfavorable_label=1.0,
)

# Initialize DisparateImpactRemover 
di_remover = DisparateImpactRemover(repair_level=1)
binaryLabelDataset_transform = di_remover.fit_transform(binaryLabelDataset_train)
df_transform = binaryLabelDataset_transform.convert_to_dataframe()[0]


# Make sure to repeat above steps for test dataset as well!

In [16]:
# Alternatively if you want to build the model with reweighting, you can use the custom function below        
def reweighing(data, label, sensitive_attr, return_list=True):
    label_dict = dict()
    try:
        # This will loop through the different labels (1 - awarded grant, 0 - not awarded)
        for outcome in data[label].unique():
            weight_map = dict()
            # Check for all possible groups (here we have A & B but there could be more in reality)
            for val in data[sensitive_attr].unique():
                # Calculate the probabilities
                nom = (
                    len(data[data[sensitive_attr] == val])
                    / len(data)
                    * len(data[data[label] == outcome])
                    / len(data)
                )
                denom = len(
                    data[(data[sensitive_attr] == val) & (data[label] == outcome)]
                ) / len(data)
                # Store weights according to sensitive attribute
                weight_map[val] = round(nom / denom, 2)
            # Store
            label_dict[outcome] = weight_map
        # Create full list of all weights for every data point provided as input
        data["weights"] = list(
            map(lambda x, y: label_dict[y][x], data[sensitive_attr], data[label])
        )
        if return_list == True:
            return data["weights"].to_list()
        else:
            return label_dict
    except Exception as err:
        print(err)
        print("Dataframe might have no entries.")
        
reweigh_dict = reweighing(training_data, 'credit_risk', 'age_groups', False)

### 2.3 <a name="23">Train - Validation Datasets</a>
(<a href="#2">Go to Data Processing</a>)

We already have training and test datasets, however the test dataset is missing the labels - the goal of the project is to predict these labels. 

To produce a validation set to evaluate model performance, split the training dataset into train and validation subsets using sklearn's [train_test_split()](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) function. Validation data you get here will be used later in section 3 to tune your classifier.

In [17]:
# Implement here


In [18]:
# We only need to split between train and val (test is already separate)
train_data, val_data = train_test_split(
    training_data, test_size=0.1, shuffle=True, random_state=23
)

# Print the shapes of the Train - Test Datasets
print(
    "Train - Test - Validation datasets shapes: ",
    train_data.shape,
    test_data.shape,
    val_data.shape,
)

Train - Test - Validation datasets shapes:  (720, 25) (200, 23) (80, 25)


### 2.5 <a name="25">Data processing with Pipeline</a>
(<a href="#2">Go to Data Processing</a>)

Build a [pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html)  to impute the missing values and scale the numerical features, and finally train a [LogisticRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)  on the imputed and scaled dataset. 


In [19]:
# Implement here


In [20]:
### STEP 1 ###
##############

# Preprocess the numerical features
numerical_processor = Pipeline(
    [("num_imputer", SimpleImputer(strategy="mean")), ("num_scaler", MinMaxScaler())]
)
# Preprocess the categorical features
categorical_processor = Pipeline(
    [
        ("cat_imputer", SimpleImputer(strategy="constant", fill_value="missing")),
        ("cat_encoder", OneHotEncoder(handle_unknown="ignore")),
    ]
)

### STEP 2 ###
##############

# Combine all data preprocessors from above
data_processor = ColumnTransformer(
    [
        ("numerical_processing", numerical_processor, numerical_features),
        ("categorical_processing", categorical_processor, categorical_features),
    ]
)

### STEP 3 ###
##############

# Pipeline desired all data transformers, along with an estimator at the end
# Later you can set/reach the parameters using the names issued - for hyperparameter tuning, for example
pipeline = Pipeline(
    [
        ("data_processing", data_processor),
        ("lg", LogisticRegression(solver="lbfgs", penalty="none")),
    ]
)

## 3. <a name="3">Train (and Tune) a Classifier</a> (Implement)
(<a href="#0">Go to top</a>)

Train (and tune) the [LogisticRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) pipeline. For tuning, you can try different imputation strategoes, different scaling methods.

In [21]:
# Implement here


In [22]:
# Get train data to train the classifier
X_train = train_data[model_features]
y_train = train_data[model_target]

# Fit the classifier to the train data
# Train data going through the Pipeline it's imputed (with means from the train data),
#   scaled (with the min/max from the train data),
#   and finally used to fit the model

train_sample_weights = list(map(lambda x,y: reweigh_dict[y][x], train_data['age_groups'], train_data[model_target]))

pipeline.fit(X_train, y_train, lg__sample_weight = train_sample_weights);

## 4. <a name="4">Make Predictions on the Test Dataset</a> (Implement)
(<a href="#0">Go to top</a>)

Use the trained classifier to predict the labels on the test set. Below you will find a code snippet that evaluates for DI.

In [23]:
# Implement here

# Get test data to test the classifier
# ! test data should come from german_credit_test.csv !
# ...

# Use the trained model to make predictions on the test dataset
# test_predictions = ...


In [24]:
# Get validation data to validate the classifier
X_test = test_data[model_features]

# Use the fitted model to make predictions on the test dataset
# Test data going through the Pipeline it's imputed (with means from the train data),
#   scaled (with the min/max from the train data),
#   and finally used to make predictions
test_predictions = pipeline.predict(X_test)

## 5. <a name="5">Evaluate Results</a> (Given)
(<a href="#0">Go to top</a>)

In [ ]:
import pandas as pd

result_df = pd.DataFrame(columns=["ID", "credit_risk"])
result_df["ID"] = test_data["ID"].tolist()
result_df["credit_risk"] = test_predictions

result_df.to_csv("../../data/final_project/project_day2_result.csv", index=False)

### Final Evaluation on Test Data - Disparate Impact
To evaluate the fairness of the model predictions, we will calculate the disparate impact (DI) metric. For more details about DI you can have a look [here](https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-post-training-bias-metric-di.html).

In [ ]:
def calculate_di(test_data, pred_df):
    """
    Function to calculate Disparate Impact metric using the results from this notebook.
    """
    try:
        # Merge predictions with original test data to model per group
        di_df = pred_df.merge(test_data, on='ID')
        # Count for group with members less than 25y old 
        pos_outcomes_less25 = di_df[di_df['age_groups']==0]['credit_risk_pred'].value_counts()[0] # value_counts()[0] takes the count of the '0 credit risk' == 'not credit risk'
        total_less25 = len(di_df[di_df['age_groups']==0])
        # Count for group with members greater equal 25y old
        pos_outcomes_geq25 = di_df[di_df['age_groups']==1]['credit_risk_pred'].value_counts()[0] # value_counts()[0] takes the count of the '0 credit risk' == 'not credit risk'
        total_geq25 = len(di_df[di_df['age_groups']==1])
        # Check if correct number of gorups
        if total_geq25 == 0:
            print('There is only one group present in the data.')
        elif total_less25 == 0:
            print('There is only one group present in the data.')
        else:
            disparate_impact = (pos_outcomes_less25/total_less25)/(pos_outcomes_geq25/total_geq25)
            return disparate_impact
    except:
        print("Wrong inputs provided.")


In [ ]:
calculate_di(test_data, result_df)

### Final Evaluation on Test Data - Accuracy & F1 Score
In addition to fairness evaluation, we also need to check the general model performance. During the EDA stage we learnt that the target distribution is skewed so we will use F1 score in addition to accuracy.

In [ ]:
accuracy_score(pd.read_csv('../../data/final_project/german_credit_test_labels.csv')['credit_risk'], result_df['credit_risk_pred'])

In [ ]:
f1_score(pd.read_csv('../../data/final_project/german_credit_test_labels.csv')['credit_risk'], result_df['credit_risk_pred'])

## Appendix

Careful if you plan to use AIF360 in the `Feature Transformation` section: You will have to transform the columns back to their original data types before passing them into the Sklearn pipeline. Also, you will have to apply the transformation to the test dataset seperatly as you are working with data that has already been split. Any transformation we apply (such as DI removal) needs to be added to all data set splits. Also, AIF360 expects to receive numerical columns only, so you will need to apply the data processor to convert your dataframe. In summary (if you plan to use AIF360):
1. Convert your data to numerical values only
2. Create BinaryLabelDataset
3. Apply transformation to all splits